core5_06_submission_selector.ipynb

목적:
1) 분석 레포에서 쌓인 실패 실험 중,
   제출용으로 '남길 것 / 버릴 것'을 선별한다.
2) Core5 전체 산출물이 재현 가능한 상태로 존재하는지
   최종 freeze 체크를 수행한다.

주의:
- 결론을 쓰지 않는다.
- '선별 기준'과 '선택 결과'만 남긴다.

In [6]:
from pathlib import Path
import pandas as pd
import json

CORE5_DIR = Path("../artifact/core5")

FAILURE_MATRIX_PATH = CORE5_DIR / "failure_matrix.csv"
FAILURE_CATALOG_PATH = CORE5_DIR / "failure_catalog.json"

failure_matrix = pd.read_csv(FAILURE_MATRIX_PATH)

with open(FAILURE_CATALOG_PATH, "r") as f:
    failure_catalog = json.load(f)

failure_matrix.head(), failure_catalog.keys()

SUBMISSION_CRITERIA = {
    "keep_if": [
        "구조 메시지가 단일하게 드러난다",
        "실험 설정이 직관적이다",
        "한 장으로 설명 가능하다"
    ],
    "drop_if": [
        "파라미터 조합이 과도하게 많다",
        "설명에 전제 설명이 길다",
        "구조 메시지가 겹친다"
    ]
}

SUBMISSION_CRITERIA

{'keep_if': ['구조 메시지가 단일하게 드러난다', '실험 설정이 직관적이다', '한 장으로 설명 가능하다'],
 'drop_if': ['파라미터 조합이 과도하게 많다', '설명에 전제 설명이 길다', '구조 메시지가 겹친다']}

In [7]:
complex_experiments = failure_matrix[
    (failure_matrix["lookback_window"] > 5) |
    (failure_matrix["cooldown"] > 2) |
    (failure_matrix["candidate_sample_n"] > 10)
]["exp_id"].unique().tolist()

complex_experiments #  failure_matrix 기반 1차 필터링

# 제출용에서 가장 설명력 높은 실패 유형
PRIORITY_FAILURE_TYPES = [
    "T1_Boundary_Oscillation",
    "T3_Burst_Cascade",
    "T5_Rule_Interaction_Explosion"
]

priority_entries = []

for entry in failure_catalog["entries"]:
    if any(t in PRIORITY_FAILURE_TYPES for t in entry["failure_types"]):
        priority_entries.append(entry)

priority_entries

for entry in failure_catalog["entries"]:
    if any(t in PRIORITY_FAILURE_TYPES for t in entry["failure_types"]):
        priority_entries.append(entry)

priority_entries # failure_catalog 기반 “설명력 높은 유형” 추출

[{'antibody_key': 'GDPa1-001',
  'failure_types': ['T1_Boundary_Oscillation',
   'T3_Burst_Cascade',
   'T4_Recovery_Overshoot'],
  'toggle_rate': 0.45,
  'burst_max': 3,
  'example_steps': {'T1_Boundary_Oscillation': [1, 4, 7],
   'T3_Burst_Cascade': [1, 3, 1],
   'T4_Recovery_Overshoot': [4, 5, 6]}},
 {'antibody_key': 'GDPa1-045',
  'failure_types': ['T1_Boundary_Oscillation',
   'T3_Burst_Cascade',
   'T4_Recovery_Overshoot'],
  'toggle_rate': 0.3,
  'burst_max': 6,
  'example_steps': {'T1_Boundary_Oscillation': [2, 4, 5],
   'T3_Burst_Cascade': [2, 1, 6],
   'T4_Recovery_Overshoot': [1, 4, 8]}},
 {'antibody_key': 'GDPa1-183',
  'failure_types': ['T1_Boundary_Oscillation',
   'T3_Burst_Cascade',
   'T4_Recovery_Overshoot'],
  'toggle_rate': 0.35,
  'burst_max': 3,
  'example_steps': {'T1_Boundary_Oscillation': [1, 8, 9],
   'T3_Burst_Cascade': [1, 1, 3],
   'T4_Recovery_Overshoot': [8, 13, 14]}},
 {'antibody_key': 'GDPa1-001',
  'failure_types': ['T1_Boundary_Oscillation',
   'T3_Bu

In [11]:
# 단순 규칙:
# - priority failure type 포함
# - 복잡 실험(exp_id) 제외
# - 항체당 1개만 선택

selected_for_submission = []
used_antibodies = set()

for entry in priority_entries:
    ab = entry["antibody_key"]
    exp_id = entry.get("exp_id")

    if ab in used_antibodies:
        continue
    if exp_id in complex_experiments:
        continue

    selected_for_submission.append({
        "exp_id": exp_id,
        "antibody_key": ab,
        "failure_types": entry["failure_types"],
        "reason": "구조 메시지가 단일하고 대표성이 높음"
    })
    used_antibodies.add(ab)

    if len(selected_for_submission) == 3:
        break

selected_for_submission

dropped_experiments = []

for _, row in failure_matrix.iterrows():
    if row["exp_id"] in complex_experiments:
        dropped_experiments.append({
            "exp_id": row["exp_id"],
            "antibody_key": row["antibody_key"],  # ✅ 추가
            "reason": "파라미터 조합 과다로 설명 복잡"
        })

dropped_experiments[:5] # 버릴 실험 목록 + 이유 생성

[{'exp_id': 'EXP_003',
  'antibody_key': 'GDPa1-001',
  'reason': '파라미터 조합 과다로 설명 복잡'},
 {'exp_id': 'EXP_003',
  'antibody_key': 'GDPa1-045',
  'reason': '파라미터 조합 과다로 설명 복잡'},
 {'exp_id': 'EXP_003',
  'antibody_key': 'GDPa1-183',
  'reason': '파라미터 조합 과다로 설명 복잡'},
 {'exp_id': 'EXP_006',
  'antibody_key': 'GDPa1-001',
  'reason': '파라미터 조합 과다로 설명 복잡'},
 {'exp_id': 'EXP_006',
  'antibody_key': 'GDPa1-045',
  'reason': '파라미터 조합 과다로 설명 복잡'}]

In [12]:
KEY_SENTENCE_CANDIDATES = [
    "규칙을 강화해도 의사결정 불안정성은 사라지지 않았다.",
    "실패는 예측 정확도의 문제가 아니라 구조적 상태 부재에서 발생했다.",
    "규칙 기반 설계는 상태 계측 없이 안정화될 수 없음을 확인했다."
]

KEY_SENTENCE_CANDIDATES

md_lines = []

md_lines.append("# Submission Selection Summary\n")

md_lines.append("## ✅ Experiments to Keep\n")
for item in selected_for_submission:
    md_lines.append(
        f"- **{item['exp_id']} / {item['antibody_key']}**: "
        f"{', '.join(item['failure_types'])} → {item['reason']}"
    )

md_lines.append("\n## ❌ Experiments to Drop\n")
for item in dropped_experiments:
    md_lines.append(
        f"- **{item['exp_id']} / {item['antibody_key']}**: {item['reason']}"
    )

md_lines.append("\n## 🧠 Key Sentence Candidates\n")
for s in KEY_SENTENCE_CANDIDATES:
    md_lines.append(f"- {s}")

submission_md = "\n".join(md_lines)
submission_md

OUTPUT_MD_PATH = CORE5_DIR / "what_to_keep_for_submission.md"

with open(OUTPUT_MD_PATH, "w") as f:
    f.write(submission_md)

OUTPUT_MD_PATH

REQUIRED_FILES = [
    "core5_inputs_manifest.json",
    "recomputed_failure_metrics.csv",
    "failure_matrix.csv",
    "counterfactual_results.csv",
    "failure_catalog.json",
    "what_to_keep_for_submission.md"
]

freeze_status = {}

for fname in REQUIRED_FILES:
    path = CORE5_DIR / fname
    freeze_status[fname] = path.exists()

freeze_status

freeze_summary = {
    "core5_status": "PASS" if all(freeze_status.values()) else "FAIL",
    "checked_files": freeze_status,
    "note": "Core5 실패 실험이 선별 기준과 함께 재현 가능 상태로 고정됨"
}

FREEZE_OUT_PATH = CORE5_DIR / "core5_debug_freeze_summary.json"

with open(FREEZE_OUT_PATH, "w") as f:
    json.dump(freeze_summary, f, indent=2)

FREEZE_OUT_PATH, freeze_summary

(PosixPath('../artifact/core5/core5_debug_freeze_summary.json'),
 {'core5_status': 'PASS',
  'checked_files': {'core5_inputs_manifest.json': True,
   'recomputed_failure_metrics.csv': True,
   'failure_matrix.csv': True,
   'counterfactual_results.csv': True,
   'failure_catalog.json': True,
   'what_to_keep_for_submission.md': True},
  'note': 'Core5 실패 실험이 선별 기준과 함께 재현 가능 상태로 고정됨'})